In [ ]:
!pip install -q -U git+https://github.com/fabienfrfr/tptt@main

In [ ]:
# only in kaggle for HF
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

In [ ]:
base_model_name="meta-llama/Llama-3.2-1B"
tokenizer_name=base_model_name

# saving
username = "ffurfaro"
model_name = '/Titans-' + base_model_name.split('/')[1]
repo_id = username + '/' + model_name.lstrip('/')

In [ ]:
import tptt
from tptt.modeling_tptt import TpttModel

model_tptt = TpttModel.from_pretrained(repo_id, token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(repo_id, token=hf_token)

In [ ]:
device = 0 if torch.cuda.is_available() else -1
model_tptt.to(f"cuda:{device}" if device != -1 else "cpu")
model_tptt.eval()

pipe = tptt.TpttPipeline(model=model_tptt.backbone, tokenizer=tokenizer, device=None)

result = pipe("Bonjour, I'm Fabien Furfaro, ", max_new_tokens=512)
print(result[0]["generated_text"],'\n', len(result[0]["generated_text"]))